# Automation of Accounting Reports

### Python script to automate repetitive tasks for  financial audits

This python will perform the following tasks:
- Open every Excel Report located in a folder
- Processing and cleaning data
- Build the monthly report following a specific format
- Merge the monthly report with the global data frame
- Saving final results in an Excel file

## 1. Define the list of excel files to process and the final report dataframe

In [9]:
import pandas as pd

# Dictionary of filenames 
list_files = ['DC-JAN-2017.xlsx', 'DC-FEB-2017.xlsx', 'DC-MAR-2017.xlsx','DC-APR-2017.xlsx', 'DC-MAY-2017.xlsx', 'DC-JUN-2017.xlsx',
             'DC-JUL-2017.xlsx', 'DC-AUG-2017.xlsx', 'DC-SEP-2017.xlsx','DC-OCT-2017.xlsx', 'DC-NOV-2017.xlsx', 'DC-DEC-2017.xlsx']
zip_loop = zip(list_files, [i[3:6] for i in list_files])

In [10]:
# Final report DataFrame
df_report = pd.DataFrame()

## 2. Create a function that opens the excel file and cleans it

In [11]:
# Function to open and clean
def clean(file_raw, month):
    # Open the file and start from line 5
    df_raw = pd.read_excel(file_raw, header = 5)
    df_raw.head()

    # Remove First Lines
    df_clean = df_raw.copy()
    df_clean = df_clean.iloc[4:]

    # FillNa with '-' (strings)
    for col in df_raw.columns[0:2]:
        df_clean[col] = df_clean[col].fillna('-')

    # FillNa with 0 (numeric)
    for col in df_raw.columns[2:]:
        df_clean[col] = df_clean[col].fillna(0).round(1)

    # TRIM Column values 
    df_clean.columns = [str(t).strip() for t in df_clean.columns]

    return df_clean 

## 3. Create a function that processes the cleaned dataframe and reformats it 

In [ ]:
# Function to process and reformat the dataframe
def process_month(df_clean, month):

    # Type
    df_clean['Type'] = df_clean[['Renting', 'Investment']].apply(
        lambda t: 'Rent.' if t['Renting']=='X' else 'Invest.' if t['Investment']=='X' else 'Purch.', axis = 1)

    # Quantity 
    dict_qty = dict(zip(['Rent.', 'Purch.', 'Invest.'], ['Rental Units', 'Purchasing Units', 'Invests. Units']))
    df_clean['Qty'] = df_clean.apply(lambda t: t[dict_qty[t['Type']]], axis = 1)

    #  Unit Cost  
    dict_cost = dict(zip(['Rent.', 'Purch.', 'Invest.'], ['Unit Rental Cost per month', 'Purchasing Unit Cost', 'Invests. Unit Cost']))
    df_clean['Unit Cost'] = df_clean.apply(lambda t: t[dict_cost[t['Type']]], axis = 1)

    # Report dataframe
    df_report = df_clean[df_clean.columns[-3:]].copy()
    df_report.columns = [Month + '-' + str(i) for i in df_report.columns]

    # Add Month
    df_month = pd.DataFrame([pd.Series(['', 'May', ''])])
    df_month.columns = df_report.columns

    # Concat
    df_report = pd.concat([df_month, df_report], ignore_index=False)
    
    # Reset index 
    df_report.reset_index(inplace= True)
    
    return df_report 

## 4. Loop through the excel files list and for each month, open and clean the file, process and reformat it, merge it to the global dataframe and finally save everything to an excel file.

In [ ]:


for file_name, month in zip_loop:
    # Import and Clean Data
    df_clean = clean(file_raw, month)
    # Build Monthly report
    df_month = process_month(df_clean, month)
    # Merge with previous Months report
    if df_report.empty
        df_report = df_month
    else:
        df_report = df_report.merge(df_month, on = 'index')
        
# Save Final Report
df_report.to_excel('Final Report.xlsx')